# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** 
The model is composed of two parts:

- An Encoder wich uses pre-trained ResNet50 network to extract features from the images. The feature volume is passed thru a fully connected layer with the same number of nodes as the word embedding used in the second part below. This allows to combine extracted spacial features from the images and the sampled captions during training. I added a Batchnorm1d layer at the end.
- A Decoder primarily composed of a LSTM network. The decoder takes on the sample captions during the training phase and generates a caption for a new image during inference, 'decoding' the proposed features extracted from the image. A fully connected layer at the end allows to map the hidden space of the LSTM to the vocab space, producing a prediction for each word sequentially.

in details:
Encoder: Pre-trained ResNet > FC layer with embed_size > batchnorm1d. No change apart adding a BarchNorm1d layer thinking this could help equalizing the features' importance.

Decoder: Embeddings (Embed size) > LSTM (1 layer, hidden dim and drop out) > DropOut(0.5) > FC layer (Vocab size)

Based on previous literature ("Show and Tell"), I used the following hyperparameters:
- word Embedding dimension of 512
- Hidden dimension of 512
- Vocab threshold of 5 (vocabulary composed by the words with 5 occurences or more in the training dataset). Total vocabulary of 8.855 words.

I decided on these myself:
- Batch size of 256. I understand bigger batch often helps training as it reduces noise. Therefore i started with the upper range (I usually use 64 or 128).
- One LSTM layer (with the idea to start small and see how the model performs)

Overall the model has 12.227.735 learnable parameters (the pre-trained CNN extractor is fixed). One step training took 5.1 sec on average. 1 Epoch 2h15.

### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** 
I kept the proposed transform as-is. I saw horizontal flip was included which provides some data augmentation. Image are resized to the expected dimensions of Resnet. I did not want to add additional data augmentation in order NOT to penalize the feature extraction which has a strong link to the captions. For example, changing colors or intensity could affect the features. Same with random rotation or shift. Since this is not a classification task here, i felt the proposed transform was appropriate.

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** 
I selected all Decoder parameters which is a no brainer.
In addition I added the fully connected layer from the Encoder as well as the batchNorm1d.
This represents just over 12Mio parameters with my settings. These represents all the parameters required to learn the mapping between the spacial representation from the features and the words to compose the captions. This of course includes the word embedding which is learned in the process.
I left unchanged the pre-trained weights of the feature extractor in the Encoder to benefit from transfer learning.

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** 
I used Adam optimizer with standard parameters (lr=0.001, betas=(0.9,0.999)). I initally used SGD as per the paper but moved to Adam while refining the model.
I opted for Adam which is reputed to perform well on large amount of data. My initial intention was to test SGD as per the "show and attend" paper but, given the intensive training time, i switched to Adam which appeared a safer bet.

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader

import math
!pip install nltk
import nltk
nltk.download('punkt')


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 256          # batch size
vocab_threshold = 5      # minimum word count threshold
vocab_from_file = True   # if True, load existing vocab file
embed_size = 512         # dimensionality of image and word embeddings
hidden_size = 512        # number of features in hidden state of the RNN decoder
num_epochs = 1                      # number of training epochs
save_every = 1                      # determines frequency of saving model weights
print_every = 100                   # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [28]:
# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)
print('vocab_size: ', vocab_size)



Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...


100%|██████████| 414113/414113 [01:33<00:00, 4432.37it/s]


Done (t=0.90s)
creating index...
index created!
Obtaining caption lengths...
vocab_size:  8855


In [8]:
# Watch for any changes in model.py, and re-load it automatically.
% load_ext autoreload
% autoreload 2
from model import EncoderCNN, DecoderRNN

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(encoder.embed.parameters()) + list(encoder.bn.parameters()) + list(decoder.parameters())

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params, lr=0.001, betas=(0.9, 0.999), eps=1e-08)   # "Show & Tell" paper uses SGD with no momentum

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [6]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print('total number of trainable parameters: ', count_parameters(encoder)+ count_parameters(decoder))

total number of trainable parameters:  12227735


In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [10]:
import torch
torch.cuda.get_device_name(0)

'Tesla K80'

In [32]:
import os
###########################################################################
# OPTIONAL FOR FINE-TUNING ---- UPDATE FILENAMES FOR BOTH MODELS BEFORE USE
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', 'encoder-2.pkl')))
decoder.load_state_dict(torch.load(os.path.join('./models', 'decoder-2.pkl')))
###########################################################################

In [33]:
optimizer = torch.optim.Adam(params, lr=0.001, betas=(0.9, 0.999), eps=1e-08)
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

In [34]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time
from workspace_utils import active_session

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})
with active_session():
    for epoch in range(1, num_epochs+1):
        
        start_time = time.time()

        for i_step in range(1, total_step+1):

            if time.time() - old_time > 60:
                old_time = time.time()
                requests.request("POST", 
                                 "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                                 headers={'Authorization': "STAR " + response.text})

            # Randomly sample a caption length, and sample indices with that length.
            indices = data_loader.dataset.get_train_indices()
            # Create and assign a batch sampler to retrieve a batch with the sampled indices.
            new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
            data_loader.batch_sampler.sampler = new_sampler

            # Obtain the batch.
            images, captions = next(iter(data_loader))

            # Move batch of images and captions to GPU if CUDA is available.
            images = images.to(device)
            captions = captions.to(device)

            # Zero the gradients.
            decoder.zero_grad()
            encoder.zero_grad()

            # Pass the inputs through the CNN-RNN model.
            features = encoder(images)
            outputs = decoder(features, captions)

            # Calculate the batch loss.
            loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))

            # Backward pass.
            loss.backward()

            # Update the parameters in the optimizer.
            optimizer.step()

            # Get training statistics.
            current_time = time.time()
            stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f, training_time: %d hr %d min' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()), (current_time-start_time)//3600, (current_time-start_time)//60-((current_time-start_time)//3600)*60)
            previous_time = time.time()
            
            # Print training statistics (on same line).
            print('\r' + stats, end="")
            sys.stdout.flush()

            # Print training statistics to file.
            f.write(stats + '\n')
            f.flush()

            # Print training statistics (on different line).
            if i_step % print_every == 0:
                print('\r' + stats)

        # Save the weights.
        if epoch % save_every == 0:
            torch.save(decoder.state_dict(), os.path.join('./models', 'decoder3-%d.pkl' % epoch))
            torch.save(encoder.state_dict(), os.path.join('./models', 'encoder3-%d.pkl' % epoch))

# Close the training log file.
f.close()

Epoch [1/1], Step [100/1618], Loss: 2.7275, Perplexity: 15.2948, training_time: 0 hr 8 min
Epoch [1/1], Step [200/1618], Loss: 2.2485, Perplexity: 9.4735, training_time: 0 hr 17 minn
Epoch [1/1], Step [300/1618], Loss: 2.5223, Perplexity: 12.4574, training_time: 0 hr 25 min
Epoch [1/1], Step [400/1618], Loss: 2.3155, Perplexity: 10.1296, training_time: 0 hr 34 minn
Epoch [1/1], Step [500/1618], Loss: 2.2457, Perplexity: 9.4473, training_time: 0 hr 42 minn
Epoch [1/1], Step [600/1618], Loss: 2.4271, Perplexity: 11.3261, training_time: 0 hr 51 min
Epoch [1/1], Step [700/1618], Loss: 2.2890, Perplexity: 9.8648, training_time: 1 hr 0 minnn
Epoch [1/1], Step [800/1618], Loss: 2.3606, Perplexity: 10.5977, training_time: 1 hr 8 min
Epoch [1/1], Step [900/1618], Loss: 2.2604, Perplexity: 9.5867, training_time: 1 hr 17 minn
Epoch [1/1], Step [1000/1618], Loss: 2.1931, Perplexity: 8.9633, training_time: 1 hr 25 min
Epoch [1/1], Step [1100/1618], Loss: 2.2251, Perplexity: 9.2544, training_time: 1

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.

In [1]:
import os
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO

# initialize COCO API for instance annotations
dataDir = '/opt/cocoapi'
dataType = 'val2014'
instances_annFile = os.path.join(dataDir, 'annotations/instances_{}.json'.format(dataType))
coco = COCO(instances_annFile)

# initialize COCO API for caption annotations
captions_annFile = os.path.join(dataDir, 'annotations/captions_{}.json'.format(dataType))
coco_caps = COCO(captions_annFile)

loading annotations into memory...
Done (t=6.28s)
creating index...
index created!
loading annotations into memory...
Done (t=0.47s)
creating index...
index created!


In [2]:
batch_size = 1          # batch size
vocab_threshold = 5      # minimum word count threshold
vocab_from_file = True   # if True, load existing vocab file
embed_size = 512         # dimensionality of image and word embeddings
hidden_size = 512        # number of features in hidden state of the RNN decoder
num_epochs = 1                      # number of training epochs

In [25]:
% load_ext autoreload
% autoreload 2
!pip install nltk
import nltk
nltk.download('punkt')
from torchvision import transforms
from data_loader_val import get_loader_val

# (Optional) TODO #2: Amend the image transform below.
transform_val = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
val_data_loader = get_loader_val(transform=transform_val,
                                 mode='val',
                                 batch_size=batch_size,
                                 vocab_threshold=vocab_threshold,
                                 vocab_from_file=vocab_from_file,
                                 num_workers = 1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...


  0%|          | 856/202654 [00:00<00:46, 4324.26it/s]

Done (t=0.44s)
creating index...
index created!
Obtaining caption lengths...


100%|██████████| 202654/202654 [00:46<00:00, 4348.85it/s]


In [4]:
# The size of the vocabulary.
vocab_size = len(val_data_loader.dataset.vocab)
print('vocab_size: ', vocab_size)

vocab_size:  8855


In [5]:
import torch
from model import EncoderCNN, DecoderRNN

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.torch/models/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [00:06<00:00, 16750168.62it/s]


DecoderRNN(
  (embedding): Embedding(8855, 512)
  (lstm): LSTM(512, 512, batch_first=True)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=512, out_features=8855, bias=True)
)

In [6]:
import os
###########################################################################
# OPTIONAL FOR FINE-TUNING ---- UPDATE FILENAMES FOR BOTH MODELS BEFORE USE
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', 'encoder-2.pkl')))
decoder.load_state_dict(torch.load(os.path.join('./models', 'decoder-2.pkl')))
###########################################################################

In [47]:
import math
total_step = min(math.ceil(len(val_data_loader.dataset.caption_lengths) / val_data_loader.batch_sampler.batch_size), 1000)
print('total_steps: ',total_step)

total_steps:  1000


In [26]:
def clean_sentence(output):
    tokens = [val_data_loader.dataset.vocab.idx2word[x] for x in output if x not in [0, 1]]
    sentence = ' '.join(tokens[:-1])
    return sentence

In [43]:
from workspace_utils import active_session
import torch.utils.data as data
import numpy as np
import requests
from tqdm import tqdm

with active_session():
    
    encoder.eval()
    decoder.eval()
    
    pred_captions=dict()
    true_captions = dict()
    
    for i_step in tqdm(range(1, total_step+1)):
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = val_data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        val_data_loader.batch_sampler.sampler = new_sampler

        # Obtain the batch.
        image, _, ids = next(iter(val_data_loader))
        
        #caption = truth[0]

        # Move batch of images and captions to GPU if CUDA is available.
        image = image.to(device)
        # Pass the inputs through the CNN-RNN model.
        features = encoder(image)
        
        # Get the generated caption with tokens
        output = decoder.sample(features)
        # Convert tokens to a clean sentence
        sentence = clean_sentence(output)
        # assign generated clean caption to the caption output list
        pred_captions[ids.item()] = sentence[:-1]


        # Get training statistics.
        stats = 'Step [%d/%d]' % (i_step, total_step)

    encoder.train()
    decoder.train()

100%|██████████| 1000/1000 [03:38<00:00,  4.59it/s]


In [44]:
import json
# save evaluation results to ./results folder
# json.dump(pred_captions, open(predictions, 'w'))
json.dump(pred_captions, open('predictions', 'w'))
json.dump(true_captions, open('ground_truth', 'w'))

In [29]:
def get_captions(key):
    annIds = coco_caps.getAnnIds(key);
    anns = coco_caps.loadAnns(annIds)
    captions=[]
    for i in range(len(anns)):
        captions.append(anns[i]['caption'].split(' '))
    return captions

In [45]:
predicted = [pred_captions[key].split(' ') for key in pred_captions.keys()]

actual=[]
for key in pred_captions.keys():
    actual.append(get_captions(key))

In [46]:
# Calculate BLEU-4 scores
from nltk.translate.bleu_score import corpus_bleu

print('BLEU-1: %.2f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
print('BLEU-2: %.2f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
print('BLEU-3: %.2f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
print('BLEU-4: %.2f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

BLEU-1: 0.51
BLEU-2: 0.33
BLEU-3: 0.25
BLEU-4: 0.13
